<a href="https://colab.research.google.com/github/RokelisK123/gilusis/blob/main/NLPtekstosumarizeris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install transformers datasets tensorflow
import tensorflow as tf
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import pandas as pd

tf.random.set_seed(42)
np.random.seed(42)

# 1. Duomenų paruošimas


In [13]:
!wget https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv


df = pd.read_csv('news_summary_more.csv', encoding='latin-1')


df = df.dropna()
texts = df['text'].tolist()[:1000]  # Using first 1000 examples
summaries = df['headlines'].tolist()[:1000]


from sklearn.model_selection import train_test_split
train_texts, val_texts, train_summaries, val_summaries = train_test_split(
    texts, summaries, test_size=0.2, random_state=42
)

model_checkpoint = "t5-base"

--2024-12-09 09:07:26--  https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41399270 (39M) [text/plain]
Saving to: ‘news_summary_more.csv.2’

news_summary_more.c 100%[===================>]  39.48M  --.-KB/s    in 0.1s    

2024-12-09 09:07:26 (310 MB/s) - ‘news_summary_more.csv.2’ saved [41399270/41399270]



In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 512
max_target_length = 64

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [15]:
def prepare_data(texts, summaries):
    inputs = ["summarize: " + str(text) for text in texts]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding="max_length",
        return_tensors="tf"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [str(summary) for summary in summaries],
            max_length=max_target_length,
            truncation=True,
            padding="max_length",
            return_tensors="tf"
        )

    return {"input_ids": model_inputs["input_ids"],
            "attention_mask": model_inputs["attention_mask"],
            "labels": labels["input_ids"]}


batch_size = 4
train_dataset = tf.data.Dataset.from_tensor_slices(
    prepare_data(train_texts, train_summaries)
).shuffle(1000).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices(
    prepare_data(val_texts, val_summaries)
).batch(batch_size)


model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Runniname dvi epochas, kad tilpti <5 min. Atsakymas gaunamas pakankamai logiškas ir geras, tačiau visada galima padidinti epochų kiekį tikslinimui.

In [19]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2,
    verbose=1
)

Epoch 1/2
200/200 [==============================] - 128s 642ms/step - loss: 0.4152 - val_loss: 0.4024
Epoch 2/2
200/200 [==============================] - 129s 643ms/step - loss: 0.3870 - val_loss: 0.3942


In [23]:
def generate_summary(text):
    inputs = tokenizer("summarize: " + text, return_tensors="tf", max_length=max_input_length, truncation=True)
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_length=max_target_length,
        early_stopping=True,
        no_repeat_ngram_size=2,
        min_length=30,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
test_indices = np.random.randint(0, len(val_texts), 3) #naudosime tris bandymus pirma iš duomenų naujiena, tos naujienos sutrumpinimas (lyg antraštė) ir mūsų modelio sugeneruotas summary

print("Bandymo modelis su pavyzdžiais iš mūsų naujienų duomenų rinkinio:\n")
for i, idx in enumerate(test_indices, 1):
    original_text = val_texts[idx]
    actual_summary = val_summaries[idx]

    print(f"Pavyzdys {i}:")
    print("Tekstas duomenyse:", original_text[:500] + "..." if len(original_text) > 500 else original_text)
    print("\nSutrumpinimas iš duomenų:", actual_summary)
    print("\nMano modelio sugeneruotas summary:", generate_summary(original_text))
    print("\n" + "▬"*80 + "\n") #atskirsime tris bandymus kažkokiu simboliu kad būtų struktūrizuota


model.save_pretrained("./trupmodelis")
tokenizer.save_pretrained("./trupmodelis")

Bandymo modelis su pavyzdžiais iš mūsų naujienų duomenų rinkinio:

Pavyzdys 1:
Tekstas duomenyse: A 25-year-old man named Saif Ali Sharafat Ali was allegedly killed by the two brothers of his girlfriend in Mumbai on Tuesday. The police said Saif was going to marry his girlfriend, whose house he was at when her brothers attacked him. The 19-year-old and 23-year-old brothers, who were opposed to their relationship, fled after the incident but were arrested. 

Sutrumpinimas iš duomenų: 25-year-old man killed by girlfriend's brothers in Mumbai

Mano modelio sugeneruotas summary: 25-yr-old man killed by two brothers of girlfriend's brothers in Mumbai . police say he was going to marry girlfriend at house when brothers attacked him

▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬

Pavyzdys 2:
Tekstas duomenyse: Japanese fabric company Toki Sen-I Co has sued Kanye West and his fashion line Yeezy Apparel for allegedly failing to pay for an order of fleece fabri